
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 双方向リカレント ニューラル ネットワーク

:ラベル: `sec_bi_rnn`

これまでのところ、シーケンス学習タスクの実例は言語モデリングであり、シーケンス内の以前のすべてのトークンを考慮して次のトークンを予測することを目的としています。このシナリオでは、左方向のコンテキストのみを条件としたいため、標準 RNN の一方向チェーンが適切であると思われます。ただし、左方向と右方向の両方のコンテキストですべてのタイム ステップで予測を条件付けることが完全に問題ないシーケンス学習タスクのコンテキストは他にもたくさんあります。たとえば、品詞検出について考えてみましょう。特定の単語に関連付けられた品詞を評価するときに、なぜ両方向のコンテキストを考慮すべきではないのでしょうか?

もう 1 つの一般的なタスク (対象となる実際のタスクに基づいてモデルを微調整する前の事前トレーニングとして役立つことがよくあります) は、テキスト ドキュメント内のランダムなトークンをマスクしてから、シーケンス モデルをトレーニングして、次の値を予測することです。不足しているトークン。空白の後に何が来るかによって、欠落しているトークンの推定値が大幅に変化することに注意してください。
- 私は`___`です。
- 私は`___`お腹が空いています。
- 私はお腹`___`空いているので、豚を半分食べられます。

最初の文では「happy」が有力な候補のようです。 2 番目の文では「not」と「very」という単語がもっともらしく見えますが、「not」は 3 番目の文と矛盾しているように見えます。

幸いなことに、簡単な手法で単方向 RNN を双方向 RNN に変換できます (引用: `Schuster.Paliwal.1997` )。単純に、反対方向に連鎖し、同じ入力 (:numref: `fig_birnn` ) に作用する 2 つの単方向 RNN 層を実装します。最初の RNN 層の場合、最初の入力は $\mathbf{x}_1$ で、最後の入力は $\mathbf{x}_T$ ですが、2 番目の RNN 層の場合、最初の入力は $\mathbf{x}_T です。 $、最後の入力は $\mathbf{x}_1$ です。この双方向 RNN 層の出力を生成するには、基礎となる 2 つの単方向 RNN 層の対応する出力を単純に連結します。

![](http://d2l.ai/_images/birnn.svg):ラベル: `fig_birnn`

形式的には、任意のタイム ステップ $t$ に対して、ミニバッチ入力 $\mathbf{X}_t \in \mathbb{R}^{n \times d}$ を考慮します (例の数: $n$、それぞれの入力の数)例: $d$)、隠れ層活性化関数を $\phi$ とします。双方向アーキテクチャでは、このタイム ステップの前方および後方の隠れ状態は $\overrightarrow{\mathbf{H}}_t \in \mathbb{R}^{n \times h}$ および $\overleftarrow{\mathbf{それぞれ H}}_t \in \mathbb{R}^{n \times h}$ です。$h$ は隠れ単位の数です。前方および後方の非表示状態の更新は次のとおりです。

 $$ \begin{aligned} \overrightarrow{\mathbf{H}} *t &amp;= \phi(\mathbf{X}_t \mathbf{W}* {xh}^{(f)} + \overrightarrow{\mathbf{H }} *{t-1} \mathbf{W}* {hh}^{(f)} + \mathbf{b} *h^{(f)}),\ \overleftarrow{\mathbf{H}}_t &amp;= \ phi(\mathbf{X}_t \mathbf{W}* {xh}^{(b)} + \overleftarrow{\mathbf{H}} *{t+1} \mathbf{W}* {hh}^{(b) } + \mathbf{b}_h^{(b)})、\end{aligned} $$

ここで、重み $\mathbf{W *} {xh}^{(f)} \in \mathbb{R}^{d \times h}, \mathbf{W} {* hh}^{(f)} \in \ mathbb{R}^{h \times h} *, \mathbf{W} {xh}^{(b)} \in \mathbb{R}^{d \times h}, \text{ and } \mathbf{W }* {hh}^{(b)} \in \mathbb{R}^{h \times h}$、およびバイアス $\mathbf{b}_h^{(f)} \in \mathbb{R}^{ 1 \times h}$ と $\mathbf{b}_h^{(b)} \in \mathbb{R}^{1 \times h}$ はすべてモデル パラメーターです。

次に、前方および後方の隠れ状態 $\overrightarrow{\mathbf{H}}_t$ と $\overleftarrow{\mathbf{H}}_t$ を連結して、隠れ状態 $\mathbf{H}_t \in \ を取得します。 mathbb{R}^{n \times 2h}$ が出力層に供給されます。複数の隠れ層を持つ深い双方向 RNN では、そのような情報は*入力*として次の双方向層に渡されます。最後に、出力層は出力 $\mathbf{O}_t \in \mathbb{R}^{n \times q}$ (出力数: $q$) を計算します。

 $$\mathbf{O} *t = \mathbf{H}_t \mathbf{W}* {hq} + \mathbf{b}_q.$$

ここで、重み行列 $\mathbf{W}_{hq} \in \mathbb{R}^{2h \times q}$ とバイアス $\mathbf{b}_q \in \mathbb{R}^{1 \times q}$ は出力層のモデル パラメーターです。技術的には、2 つの方向で異なる数の隠れユニットを持つことができますが、この設計上の選択が実際に行われることはほとんどありません。次に、双方向 RNN の簡単な実装を示します。


In [1]:
import torch
from torch import nn
from d2l import torch as d2l


## ゼロからの実装

双方向 RNN を最初から実装するには、個別の学習可能なパラメーターを持つ 2 つの単方向`RNNScratch`インスタンスを含めることができます。


In [2]:
class BiRNNScratch(d2l.Module):
    def __init__(self, num_inputs, num_hiddens, sigma=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.f_rnn = d2l.RNNScratch(num_inputs, num_hiddens, sigma)
        self.b_rnn = d2l.RNNScratch(num_inputs, num_hiddens, sigma)
        self.num_hiddens *= 2  # The output dimension will be doubled


順方向 RNN と逆方向 RNN の状態は個別に更新され、これら 2 つの RNN の出力は連結されます。


In [3]:
@d2l.add_to_class(BiRNNScratch)
def forward(self, inputs, Hs=None):
    f_H, b_H = Hs if Hs is not None else (None, None)
    f_outputs, f_H = self.f_rnn(inputs, f_H)
    b_outputs, b_H = self.b_rnn(reversed(inputs), b_H)
    outputs = [torch.cat((f, b), -1) for f, b in zip(
        f_outputs, reversed(b_outputs))]
    return outputs, (f_H, b_H)


## 簡潔な実装



高レベル API を使用すると、双方向 RNN をより簡潔に実装できます。ここでは例として GRU モデルを取り上げます。


In [4]:
class BiGRU(d2l.RNN):
    def __init__(self, num_inputs, num_hiddens):
        d2l.Module.__init__(self)
        self.save_hyperparameters()
        self.rnn = nn.GRU(num_inputs, num_hiddens, bidirectional=True)
        self.num_hiddens *= 2


## まとめ

双方向 RNN では、各タイム ステップの隠れ状態は、現在のタイム ステップの前後のデータによって同時に決定されます。双方向 RNN は、シーケンスのエンコードと双方向コンテキストを考慮した観測値の推定に主に役立ちます。双方向 RNN は勾配チェーンが長いため、トレーニングに非常にコストがかかります。

## 演習
1. 異なる方向で使用する隠れ単位の数が異なる場合、$\mathbf{H}_t$ の形状はどのように変化しますか?
1. 複数の隠れ層を備えた双方向 RNN を設計します。
1. 多義性は自然言語では一般的です。たとえば、「銀行」という単語は、「現金を預けに銀行に行った」と「座るために銀行に行った」という文脈では意味が異なります。コンテキスト シーケンスと単語を指定すると、コンテキスト内の単語のベクトル表現が返されるようにニューラル ネットワーク モデルを設計するにはどうすればよいでしょうか?多義性を処理するにはどのタイプのニューラル アーキテクチャが推奨されますか?



[ディスカッション](https://discuss.d2l.ai/t/1059)
